In [1]:
#importing Required Libraries

import numpy as np
import os
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import csv
import matplotlib.ticker
import math

In [ ]:
#cloning the dataset from my repo
!git clone https://github.com/TensorTeen/Optiwing-Cp-Plots

In [4]:
device = 'cuda'

In [5]:
#getting file names
path = r"Optiwing-Cp-Plots"
os.chdir(path)
l = np.array([x for x in os.listdir("./")])

###Deserializing the data into arrays


In [6]:

# contains all the cp values
X = []
# Contains all the x,y values
Y = []
for i in l:
  try:
    with open(i) as f:
      try :
        next(f)
        next(f)
        next(f)
      except:
        continue
      x = []
      y = []
      cp = []
      for j in f :
        try :
          tmp = [float(j[:10]), float(j[10:19]),float(j[19:28])]
          x.append(tmp[0])
          y.append(tmp[1])
          cp.append(tmp[2])
        except :
          break
      else:
        if len(cp) == 160 :
          X.append([x,cp])
          Y.append([x,y])
  except IsADirectoryError:
      pass

###reshaping tensors for convinience


In [ ]:
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(Y)
x = X[:,0,:]
Cp = X[:,1,:]
X = torch.cat((x,Cp),dim=1)
y = y.flatten(start_dim=1)
X.shape

###assuming one of the airfoils in train set to be base airfoil

In [8]:
x_init,y_init = y[10][:160].reshape(1,160),y[10][160:].reshape(1,160)

###splitting the data into training and testing set


In [9]:

train_size = int(0.8 * len(X))
test_size = len(X) - train_size
X_train, X_test = X[:train_size],X[train_size:]
y_train, y_test = y[:train_size],y[train_size:]

###Defining the model


In [10]:
class Model(nn.Module):

    def __init__(self,n):
        super().__init__()
        self.n = n
        self.hidden1 = nn.Linear(320, 512)
        self.act1 = nn.ReLU()
        self.hidden2 = nn.Linear(512,1024)
        self.act2 = nn.ReLU()
        self.hidden3 = nn.Linear(1024,2048)
        self.act3 = nn.ReLU()
        self.hidden4 = nn.Linear(2048,1024)
        self.act4 = nn.ReLU()
        self.hidden5 = nn.Linear(1024,512)
        self.act5 = nn.ReLU()
        self.dropout = nn.Dropout(p=0.2)
        self.hidden6 = nn.Linear(512,256)
        self.act6 = nn.ReLU()
        self.hidden7 = nn.Linear(256,80)
        self.act7 = nn.ReLU()
        self.output = nn.Linear(80,n)

    def forward(self, x):
        x = self.act1(self.hidden1(x))
        x = self.act2(self.hidden2(x))
        x = self.act3(self.hidden3(x))
        x = self.act4(self.hidden4(x))
        x = self.act5(self.hidden5(x))
        x = self.dropout(x)
        x = self.act6(self.hidden6(x))
        x = self.act7(self.hidden7(x))
        x = self.output(x)
        return x


###few things being assumed:


*   all x values are same as x_init for all the airfoils




###This is the hicks function of ith order
$$f_i(x) = \sin ^{t_{i}}\left(\pi x^{\left(\ln (0.5) / \ln \left(h_{i}\right)\right)}\right) $$

In [11]:
def hicks_fni(x:torch.Tensor,i:torch.Tensor,n:int,device):
  x = x[0]
  hi = 0.5*(1-torch.cos(torch.mul(i,math.pi*(1/(n+1))))).reshape(n,1).to(device)
  out = x #check this
  out = torch.pow(out, torch.div( math.log(0.5), torch.log(hi)))
  out = torch.mul(math.pi,out)
  ti = 4
  out = torch.sin(out)
  out = torch.pow(out,ti)
  return out

###This combines all order hick functions to generate y
$$\Sigma a_i f_i(x)$$

In [12]:
def fn(x,x_init,y_init,net,device):
  y_deformed = y_init
  a = net(x)
  n = net.n
  i = torch.Tensor(range(1,n+1)).to(device)
  Del_y = torch.matmul(a,hicks_fni(x_init,i,net.n,device=device))
  y_deformed = y_deformed + Del_y
  return y_deformed

###Train Loop

In [13]:
def train(n_epochs:int, batch_size:int,model,device='cpu',lr=0.001):
    """Function used to Train the model given training parameters

    Args:
        n_epochs (int): number of epochs
        batch_size (int): Batch_size for Adam optimizer
        model (Pytorch Model): Pytorch model object to be trained
        lr (float, optional): Learning rate. Defaults to 0.001.

    Returns:
        tuple: tuple containing trained model and loss
    """
    model.train()
    loss_fn = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    global L
    L = []
    for epoch in range(n_epochs):
        for i in range(0, len(X_train), batch_size):
            global x_init
            global y_init
            x_init = x_init.to(device)
            y_init = y_init.to(device)
            Xbatch = X_train[i:i+batch_size]
            Xbatch = Xbatch.to(device)
            ybatch = y_train[i:i+batch_size][:,160:]
            ybatch = ybatch.to(device)
            y_pred = fn(Xbatch,x_init.repeat_interleave(len(Xbatch),dim=0),y_init.repeat_interleave(len(Xbatch),dim=0),model,device=device)
            loss = loss_fn(y_pred, ybatch)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        L.append(loss)
        print(f'Finished epoch {epoch}, latest loss {loss}')
    return loss,model

In [14]:
def evaluate(X_test,y_test,model):
    """Function used to evaluate the model

    Args:
        X_test (Torch.Tensor): Test Values for Cp
        y_test (Torch.Tensor): Test Values for x,y
        model (Pytorch Model): torch model to be evaluated

    Returns:
        _type_: _description_
    """
    with torch.no_grad():
        y_pred = fn(Xbatch,x_init.repeat_interleave(len(Xbatch),dim=0),y_init.repeat_interleave(len(Xbatch),dim=0),model,device=device)
    loss = (y_pred - y_test).float()**2
    loss = loss.mean()
    return loss


In [ ]:
model = Model(80)
if device == "cuda":
  model.cuda()
for parameters in model.parameters():
  parameters.requires_grad = True
loss, model = train(50,10,model,device,1e-4)

In [ ]:
torch.save(model,"/content/50_Epoch_Hicks.pth")

In [ ]:
inference_model = torch.load("/content/500_Epoch_Hicks.pth")

In [ ]:
L = [float(i) for i in L]

In [ ]:
plt.plot(range(500),L)
plt.yscale('log')
plt.title('Loss vs epoch(log scale)')

In [ ]:
with torch.no_grad():
      y_pred = fn(X_train[50].to('cuda'),x_init.repeat_interleave(1,dim=0),y_init.repeat_interleave(1,dim=0),inference_model,device='cuda')

In [ ]:
y_test[0].shape

In [ ]:
plt.plot(x_init.cpu().numpy().reshape(160),y_pred.cpu().numpy().reshape(160))
plt.plot(y_train[50][:160].cpu().numpy().reshape(160),y_train[50][160:])
plt.legend(['Parameterized','Original'])

#some observation


*   The underlying fn seems to make the trailing edge wavy in nature
*   For wings with $Δy << 1$, the bump functions seems to work well

